## Importing modules and packages

In [99]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import geopandas as gpd
from IPython.display import IFrame

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching
import gr_utils # Contains useful geometry functions
import gr_plot

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

## Input parameters

In [2]:
trailname = 'gr16' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)
delta = 0.005 # Tolerance around bounding box per trail section [deg]
points_per_batch = 100 # Subdivide the trail into batches of this many points

## Loading GPX file

In [3]:
filename_gpx = 'data_input/' + trailname + '.gpx'
filename_csv = 'data_output/' + trailname + '.csv'
if not os.path.isfile(filename_csv): # The GPX file was not processed into a clean CSV file before
    if not os.path.isfile(filename_gpx): # The GPX file does not exist, throw error
        raise ValueError(f'The GPX file <{filename_gpx}> was not found! Please make sure it exists.')
    else: # The GPX file exists, so convert it into a clean CSV file
        print(f'Converting GPX file <{filename_gpx}> into cleaned CSV file <{filename_csv}>...')
        gr_utils.process_gpx(filename_gpx,filename_csv)
        print('Completed conversion.')
print(f'Loading trail points from <{filename_gpx}>...')
trail = pd.read_csv(filename_csv) # Now read the cleaned CSV file into a DataFrame (latitude, longitude, elevation)
print('Finished loading.')

Loading trail points from <data_input/gr16.gpx>...
Finished loading.


## Matching GPX track to OSM network

In [4]:
# Matching GPX track to OSM network (uses _osm_network_download under the hood)
n_trail = len(trail) # Number of GPX points in the trail
n_batch = int(np.ceil(trail.shape[0]/points_per_batch)) # Number of batches to be run
for b in range(n_batch): # Using batch counter b
    
    # Define the range of GPX points to process in the current batch
    n1 = b*points_per_batch # First point of this batch
    n2 = min(n1 + points_per_batch, n_trail) # Last point of this batch (clipped)
    trail_section = trail.loc[n1:n2] # Select that range of GPX points
    trail_coords  = gr_mapmatch.trail_to_coords(trail_section) # Convert the points into a list of [lat, lon] pairs
    
    # Check if this batch was processed before
    batch_out = f'cache/{trailname}_{n1}to{n2}.csv'
    print(f'Handling {b} of {n_batch-1} that covers GPX track points {n1} through {n2}...')
    if os.path.isfile(batch_out): # It already exists
        print('   This batch was processed before, skipping.')
    else: # It does not exist, so process it
        network, segment_list = gr_mapmatch.match_batch(trail_section, trail_coords, delta)
        gr_utils.write_batch(batch_out, segment_list)
        print('   Finished this batch.')
        print('')

Handling 0 of 60 that covers GPX track points 0 through 100...
   This batch was processed before, skipping.
Handling 1 of 60 that covers GPX track points 100 through 200...
   This batch was processed before, skipping.
Handling 2 of 60 that covers GPX track points 200 through 300...
   This batch was processed before, skipping.
Handling 3 of 60 that covers GPX track points 300 through 400...
   This batch was processed before, skipping.
Handling 4 of 60 that covers GPX track points 400 through 500...
   This batch was processed before, skipping.
Handling 5 of 60 that covers GPX track points 500 through 600...
   This batch was processed before, skipping.
Handling 6 of 60 that covers GPX track points 600 through 700...
   This batch was processed before, skipping.
Handling 7 of 60 that covers GPX track points 700 through 800...
   This batch was processed before, skipping.
Handling 8 of 60 that covers GPX track points 800 through 900...
   This batch was processed before, skipping.
Han

## Merging sections and removing backtracks

In [12]:
filename_roads = 'cache/' + trailname + '_roads.csv'
if not os.path.isfile(filename_roads): # The merged file does not exist
    print('Merged section file was not found, merging and saving...')
    data_roads_raw = gr_utils.merge_roads(trailname, trail, points_per_batch) # Merge the different sections
    data_roads = gr_mapmatch.remove_repeat_segments(data_roads_raw) # Remove backtracked sections
    gr_utils.write_roads(trailname, data_roads) # Write the merged sections
    print('Saved.')
else: # The merged file does exist
    print('Loading merged section file...')
    data_roads = gr_utils.read_roads(trailname) # Read the merged sections
    print('Loaded.')

Loading merged section file...
Loaded.


## Gathering place information

In [92]:
def is_polygon(row):
    is_simplepoly = type(row['geometry']) is shapely.geometry.polygon.Polygon
    is_multipoly = type(row['geometry']) is shapely.geometry.multipolygon.MultiPolygon
    return is_simplepoly or is_multipoly
#     return type(row['geometry']) is shapely.geometry.polygon.Polygon

In [253]:
## Matching GPX track to OSM places (uses _osm_place_download under the hood)
points_per_batch_places = 100 # Subdivide the trail into batches of this many segments
# n1 = 0;
# n2 = points_per_batch_places;
n1 = 0*points_per_batch_places;
n2 = 5*points_per_batch_places;
delta = 0.005

# Select point subset & construct bbox
subset = data_roads.iloc[n1:n2] # subset of points dataframe
lat_min = subset['x0'].min() - delta
lat_max = subset['x0'].max() + delta
lon_min = subset['y1'].min() - delta
lon_max = subset['y1'].max() + delta
polygon = ox.utils_geo.bbox_to_poly(lat_max, lat_min, lon_max, lon_min) # polygon of bbox around subset

# Download place data with landuse tag
# tags = {"landuse": True}
tags = {"landuse": ['commercial','construction','education','industrial','residential','retail','institutional','farmyard','cemetery','garages','railway','landfill','brownfield','quarry']}
gdf = ox.geometries_from_polygon(polygon, tags)
gdf['is_polygon'] = gdf.apply(is_polygon, axis=1)
mask_polygon = gdf['is_polygon']==True
gdf_subset = gdf[mask_polygon]

In [271]:
# Test matching
developed = []
tol_d = 1.0 # Consider a segment developed if it lies closer than tol_d to a developed area
for i, segment in subset.iterrows():
    developed.append(False)
    xmid = (segment['x0'] + segment['x1'])/2
    ymid = (segment['y0'] + segment['y1'])/2
    point = shapely.geometry.Point(ymid,xmid)
    for j, area in gdf_subset.iterrows():
        d = 1000*area['geometry'].distance(point)
        if d<tol_d:
            developed[i] = True
            break;

In [272]:
perc_developed = sum(developed)/len(developed)
print(perc_developed)

0.538


In [299]:
# Create a map of the subset & polygons near it
# Map setup
chart = folium.Map(location=trail_coords[0], zoom_start=12, tiles="OpenStreetMap")

# Draw map matched frame
xy = gr_plot.get_coords_from_frame(subset)

# Draw areas
for i in range(gdf_subset.shape[0]):
    item = gdf_subset.iloc[i]
    sim_geo = gpd.GeoSeries(item['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(item['landuse']).add_to(geo_j)
    geo_j.add_to(chart)
    
for point in xy:
    newmarker = folium.CircleMarker(location=point,radius=2,color='black')
    newmarker.add_to(chart)

# Draw midpoints according to development status
for idx, row in subset.iterrows():
    x0 = row['x0']
    x1 = row['x1']
    y0 = row['y0']
    y1 = row['y1']
    xmid = (x0 + x1)/2
    ymid = (y0 + y1)/2
    midpoint = [xmid,ymid]
#     newmarker = folium.CircleMarker(location=point,radius=2,color='black')
#     newmarker.add_to(chart)
    if developed[idx]:
        newmarker = folium.CircleMarker(location=midpoint,radius=3,color='red')
    else:
        newmarker = folium.CircleMarker(location=midpoint,radius=3,color='green')
    newmarker.add_to(chart)

In [300]:
# Render the map
filepath = "cache/chart_development.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

In [296]:
# Figure out how to process landuse into a useful statistic

## Establishing paved, traffic, and development type

## Saving completed data frame

## Plotting

In [13]:
# Make a plot of the original and matched track
trail_coords  = gr_mapmatch.trail_to_coords(trail)
filepath = gr_plot.compare_tracks(trail_coords, data_roads)
IFrame(filepath, width=1000, height=500)